# &#128214; Lab 7: Call Graph

## &#128214; Exercise 1: Building a Call Graph

### &#127919; Objective
In the preceding exercises, you've delved into class hierarchy analysis and points-to analysis. 
Now, it's time to leverage the information from the points-to analysis to construct a Call Graph. 
A Call Graph illustrates the calling relationships between functions or methods in a program, which is instrumental in performing interprocedural analysis. 
The goal of this exercise is to generate a Call Graph that integrates the points-to information to achieve a more accurate representation of the possible method invocations.

### &#128214; Background
A Call Graph is a directed graph where nodes represent methods or functions, and edges signify the calling relationships. 
However, in object-oriented languages, the exact method being called may depend on the runtime type of the object. 
Points-to analysis helps to over-approximate the possible types an object variable can point to, and thereby the possible methods that can be called.

### &#10145; Tasks
1. Utilize the points-to set information from the previous exercise to determine the possible target methods for each method call.
2. Create nodes for each method and edges for each method invocation identified. An edge from method A to method B signifies that method A calls method B.

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*
- `ast`: a module of the python standard library to transform Python code in its AST representation
- `graphviz`: a library to create directed graphs


In [ ]:
import ast
import graphviz

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
class Language:
    def __init__(self):
        pass
    def say_hello(self, first_name):
        print(f"Saluton {first_name} el ĝenerala lingvo.")


class French(Language):
    def say_hello(self, first_name):
        print(f"Bonjour {first_name}.")

class English(Language):
    def say_hello(self, first_name):
        print(f"Hello {first_name}.")

class German(Language):
    def say_hello(self, first_name):
        print(f"Hallo {first_name}.")

class Spanish(Language):
    def say_hello(self, first_name):
        print(f"Hola {first_name}.")

class Italian(Language):
    def say_hello(self, first_name):
        print(f"Ciao {first_name}.")

class Portuguese(Language):
    def say_hello(self, first_name):
        print(f"Olá {first_name}.")

if x > 5:
    obj = French()
else:
    obj = English()
    
obj.say_hello("Patrick")
    
if y < 3:
    obj = German()
else:
    obj = Spanish()

obj.say_hello("John")
"""

### Helper Function Explanation

In Python, both class instantiation and function calling use the same syntax which leads to an ambiguity when analyzing the AST of a code snippet. 
Specifically, both operations appear as `ast.Call` nodes within the AST, which makes it difficult to distinguish between the two just by examining the syntax.

For instance, consider the following Python code snippet:

```python
class MyClass:
    pass

def my_function():
    pass

obj = MyClass()  # Class instantiation
my_function()    # Function call
```

In the AST, both `MyClass()` and `my_function()` will be represented as `ast.Call` nodes, which could lead to misinterpretation during analysis.

To address this issue and provide clearer analysis, we present a helper function that traverses the AST to identify and categorize all class and function definitions. 
This will later help in accurately interpreting `ast.Call` nodes.

### Class and Function Analyzer

We define a class `ClassFunctionAnalyzer` which inherits from `ast.NodeVisitor`. 
This class overrides three methods: `visit_ClassDef`, `visit_FunctionDef`, and `visit_Call`.

```python
class ClassFunctionAnalyzer(ast.NodeVisitor):
    """
    A node visitor class that walks through the Abstract Syntax Tree (AST) to identify and record all class and function definitions within the code.

    Attributes:
    classes: A set to store unique class names encountered in the AST.
    functions: A set to store unique function names encountered in the AST.
    """
    def __init__(self):
        self.classes = set()
        self.functions = set()

    def visit_ClassDef(self, node):
        """
        Visits a class definition node, adding the class name to the classes set.

        Parameters:
        node: The class definition node being visited.
        """
        self.classes.add(node.name)
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        """
        Visits a function definition node, adding the function name to the functions set.

        Parameters:
        node: The function definition node being visited.
        """
        self.functions.add(node.name)
        self.generic_visit(node)
        
    def get_call_type(self, name):
        """
        Determines whether a given name corresponds to a class instantiation or a function call.

        Parameters:
        name: The name of the function or class.

        Returns:
        str: "class" if the name corresponds to a class, "function" if it corresponds to a function, or "unknown" otherwise.
        """
        if name in self.classes:
            return "class"
        elif name in self.functions:
            return "function"
        else:
            return "unknown"
```

#### Explanations:

- `__init__`: Initializes two sets, `classes` and `functions`, to store the names of classes and functions respectively.
  
- `visit_ClassDef`: This method is invoked for every `ast.ClassDef` node encountered. It adds the name of the class to the `classes` set.

- `visit_FunctionDef`: This method is invoked for every `ast.FunctionDef` node encountered. It adds the name of the function to the `functions` set.

- `visit_Call`: This method is invoked for every `ast.Call` node encountered. It checks whether the called entity is a class or a function by comparing the identifier against the collected sets of class and function names.

By using this helper function, you will be able to determine whether an `ast.Call` node represents a class instantiation or a function call, which is important for the points-to analysis you will be performing.
Of course, this solution is not optimal/perfect since we might have different functions with the same name in a program, but for simplicity we make the hypothesis that functions are unique. 

In [ ]:
class ClassFunctionAnalyzer(ast.NodeVisitor):
    """
    A node visitor class that walks through the Abstract Syntax Tree (AST) to identify and record all class and function definitions within the code.

    Attributes:
    classes: A set to store unique class names encountered in the AST.
    functions: A set to store unique function names encountered in the AST.
    """
    def __init__(self):
        self.classes = set()
        self.functions = set()

    def visit_ClassDef(self, node):
        """
        Visits a class definition node, adding the class name to the classes set.

        Parameters:
        node: The class definition node being visited.
        """
        self.classes.add(node.name)
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        """
        Visits a function definition node, adding the function name to the functions set.

        Parameters:
        node: The function definition node being visited.
        """
        self.functions.add(node.name)
        self.generic_visit(node)
        
    def get_call_type(self, name):
        """
        Determines whether a given name corresponds to a class instantiation or a function call.

        Parameters:
        name: The name of the function or class.

        Returns:
        str: "class" if the name corresponds to a class, "function" if it corresponds to a function, or "unknown" otherwise.
        """
        if name in self.classes:
            return "class"
        elif name in self.functions:
            return "function"
        else:
            return "unknown"

### &#128161; How to use the analyzer class to use throughout this lab

In [ ]:
tree = ast.parse(code)
analyzer = ClassFunctionAnalyzer()
analyzer.visit(tree)

### &#128161; In the following cell, you will reuse the latest `build_points_to_set` you implemented (with the highest level of precision) and build the points-to set of the code provided.

In [ ]:
def build_points_to_set(node, analyzer):
    """
    Analyzes an AST to gather information about potential class instantiations and method calls.
    
    Parameters:
    node: The root node of the AST.
    analyzer: An instance of ClassFunctionAnalyzer.
    
    Returns:
    dict: The points-to set.
    """

### Utility Call Graph class

&#128161; The following cell contains a utility class to build a Call Graph. 
You have to use this class to build the call graph.

In [ ]:
class CallGraph:
    """A class to represent a call graph."""
    
    def __init__(self):
        """Initializes an empty call graph."""
        self.graph = {}
    
    def add_edge(self, caller, callee):
        """
        Adds an edge from caller to callee in the graph.
        
        Parameters:
        caller: The caller function.
        callee: The callee function.
        """
        self.graph.setdefault(caller, set()).add(callee)
    
    def visualize(self):
        """Visualizes the call graph."""
        for caller, callees in self.graph.items():
            for callee in callees:
                print(f'{caller} -> {callee}')
                
    def to_dot(self):
        """
        Converts the call graph to a dot representation.
        
        Returns:
        str: The dot representation of the call graph.
        """
        dot_lines = ["digraph CallGraph {"]
        for caller, callees in self.graph.items():
            for callee in callees:
                dot_lines.append(f'    "{caller}" -> "{callee}";')
        dot_lines.append("}")
        return '\n'.join(dot_lines)



### Instructions for Implementing a Call Graph Builder in Python

&#128161; In Python, it's not mandatory for code to be enclosed within a function. 
In this scenario, we'll assume the root to serve a similar purpose to the main method found in other programming languages.

You will be using the `CallGraphBuilder` class, a subclass of `ast.NodeVisitor`, to implement a system for constructing a call graph in Python. This class is designed to analyze Python code and build a graph that represents the relationships between function and method calls.

Key aspects to focus on:

1. **Class and Function Context**: The `CallGraphBuilder` maintains two stacks - `function_stack` and `class_stack`. The `function_stack` tracks the current function context, while the `class_stack` keeps track of the current class context. This setup helps in accurately identifying where each function or method is being called.

2. **Handling Method Calls**: The class is equipped to handle method calls (e.g., `obj.method()`). When a method call is encountered, it's processed to determine the caller and callee, and this relationship is added to the call graph.

3. **Graph Construction**: The `call_graph` attribute of the `CallGraphBuilder` instance will store the call graph. It's a `CallGraph` object that holds the edges representing the call relationships.

4. **Visiting Different Node Types**: The class overrides methods like `visit_ClassDef`, `visit_FunctionDef`, `visit_Expr`, and `visit_Assign` to handle different types of AST nodes. This ensures that all relevant Python constructs are analyzed to build the call graph.

Your task is to implement this `CallGraphBuilder` class to analyze Python source code and build a call graph. 

In [ ]:
class CallGraphBuilder(ast.NodeVisitor):
    def __init__(self, points_to_set):
        self.call_graph = CallGraph()
        self.points_to_set = points_to_set

### &#128161; In the following cell, you will test your implementation.
Build the call graph set using the `build_call_graph` function and print it using the `visualize` function.

### Visualization of the call graph

&#128161; In the following cell, you will create a `Digraph` of the `graphviz` library using the `Source()` function that takes a `dot` formatted string as a parameter, and you will display the graph as in the previous Lab.

Examples:

```python
dot_string = call_graph.to_dot()
graph = graphviz.Source(dot_string)
graph
```


Ok, now we know that, in the root function (which doesn't really exist), the say_hello method of the German, French, English, and Spanish classes is called, as well as the constructors of these classes.

Note that we only covered methods called on objects, as this is where the challenge lies.
Indeed, if a function like "print" is called, there's no need for a points-to analysis; we can directly connect the call to this function in the call graph.

### &#128161; Now, you will function calls not performed on objects to the call graph.

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
class Language:
    def __init__(self):
        pass
    def say_hello(self, first_name):
        print(f"Saluton {first_name} el ĝenerala lingvo.")


class French(Language):
    def say_hello(self, first_name):
        print(f"Bonjour {first_name}.")

class English(Language):
    def say_hello(self, first_name):
        print(f"Hello {first_name}.")

class German(Language):
    def say_hello(self, first_name):
        print(f"Hallo {first_name}.")

class Spanish(Language):
    def say_hello(self, first_name):
        print(f"Hola {first_name}.")

class Italian(Language):
    def say_hello(self, first_name):
        print(f"Ciao {first_name}.")

class Portuguese(Language):
    def say_hello(self, first_name):
        print(f"Olá {first_name}.")

if x > 5:
    obj = French()
else:
    obj = English()
    
obj.say_hello("Patrick")
    
if y < 3:
    obj = German()
else:
    obj = Spanish()

obj.say_hello("John")

print("Done")
"""

&#128161; If we consider this code and apply the previous algorithm that focused on functions called on objects, the "print" function would not appear in the call graph.

&#10145; Change the `build_call_graph` function so that it appears.

In [ ]:
class CallGraphBuilder(ast.NodeVisitor):
    def __init__(self, points_to_set):
        self.call_graph = CallGraph()
        self.points_to_set = points_to_set

### &#128161; In the following cell, you will test your implementation.
Build the call graph set using the `build_call_graph` function and print it using the `visualize` function.

### Visualization of the call graph

&#128161; In the following cell, you will create a `Digraph` of the `graphviz` library using the `Source()` function that takes a `dot` formatted string as a parameter, and you will display the graph.


Ok good! Congratulations!

But what if now I ask you to use this call graph at a specific program point, say line 38, to know what could be the possible methods called?

....

You are right, you cannot since there is not such information in the call graph.
So what would you do to solve this problem?